# Import

In [1]:
import logging
import helpers.hdbg as hdbg
import linkedin.phantom_api.phantombuster_api as lpphapia
import linkedin.google_api.google_file_api as lggogfia

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [2]:
_LOG = logging.getLogger(__name__)
hdbg.init_logger(use_exec_path=True)

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-4af962c7-a3da-4737-8623-fd7978c6fd1f.json'
INFO: Saving log to file '/app/linkedin/notebooks/none.log'


# Set search name

In [3]:
search_name = "sn_test_folder"

# Create the empty Google Drive folder and Google sheets

In [4]:
googleapi = lggogfia.GoogleFileApi()

INFO  file_cache is only supported with oauth2client<4.0.0


In [5]:
# linkedin_data folder id.
parent_folder_id = "1dQ9e-bNKkXwNvobQyRFbPwgEh1-VSf4R"

In [6]:
# Create a folder with search_name in the dir parent folder.
current_folder_id = googleapi.create_google_drive_folder(search_name, parent_folder_id)

INFO  Created a new Google Drive folder 'sn_test_folder'.
INFO  The new folder id is '14jZ9t4U9Nxx4bHix1VWGoGNodkG9-thz'.


In [7]:
# Create empty gsheets in the new created folder.
gsheets_name = [
    f"{search_name}.step1.search_export",
    f"{search_name}.step2.search_export_filtered",
    f"{search_name}.step3.profile_export",
    f"{search_name}.step4.profile_export_filtered",
]
for gsheet_name in gsheets_name:
    googleapi.create_empty_google_file(
        gfile_type = "sheet",
        gfile_name = gsheet_name,
        gdrive_folder_id = current_folder_id,
        user = ""
    )

INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_test_folder.step1.search_export'.
INFO  Move the new Google sheet 'sn_test_folder.step1.search_export' to the given dir
INFO  Finished creating the new Google sheet 'sn_test_folder.step1.search_export'.
INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_test_folder.step2.search_export_filtered'.
INFO  Move the new Google sheet 'sn_test_folder.step2.search_export_filtered' to the given dir
INFO  Finished creating the new Google sheet 'sn_test_folder.step2.search_export_filtered'.
INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_test_folder.step3.profile_export'.
INFO  Move the new Google sheet 'sn_test_folder.step3.profile_export' to the given dir
INFO  Finished creating the new Google sheet 'sn_test_folder.step3.profile_export'.
INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created 

# Download result CSVs to local storage

In [8]:
phantom = lpphapia.Phantom()

In [9]:
# Get all phantoms and their phantom id.
phantom.get_all_phantoms()

,id,name,scriptId,lastEndMessage,lastEndStatus,queuedContainers,runningContainers
0,3593602419926765,Yiyun LinkedIn Profile Scraper,3112,,success,0,0
1,2862499141527492,Yiyun Search5 Sales Navigator Search Export,6988,,success,0,0
2,3933308360008191,GP LinkedIn Profile Scraper,3112,,success,0,0
3,4170077962142115,GP Sales Navigator Search Export,6988,,success,0,0
4,4809903115550081,Retired MBA DC UMD LinkedIn Search Export,3149,,success,0,0


In [10]:
# (INPUT) Set the phantom IDs (Choose ID from the above table).
search_phantom_id = "2862499141527492"
profile_phantom_id = "3593602419926765"

In [11]:
# Path to save result csv.
result_dir = "../result_csv/"
search_result_csv_path = result_dir + f"{search_name}_search_result.csv"
profile_result_csv_path = result_dir + f"{search_name}_profile_result.csv"

In [12]:
# Download search result csv.
phantom.download_result_csv_by_phantom_id(search_phantom_id, search_result_csv_path)

INFO  Result CSV URL: https://phantombuster.s3.amazonaws.com/jqWbRHyznhM/pyjlATELCNt5qJDusZToQg/result.csv
INFO  Result CSV saved to ../result_csv/sn_test_folder_search_result.csv


In [13]:
# Download profile result csv.
phantom.download_result_csv_by_phantom_id(profile_phantom_id, profile_result_csv_path)

INFO  Result CSV URL: https://phantombuster.s3.amazonaws.com/jqWbRHyznhM/EAS5IvYVZQiP2OFtv7KqjQ/result.csv
INFO  Result CSV saved to ../result_csv/sn_test_folder_profile_result.csv


# Upload result CSVs to Google sheets

In [14]:
import gspread_pandas
import pandas as pd

In [15]:
search_export_df = pd.read_csv(search_result_csv_path)
profile_export_df = pd.read_csv(profile_result_csv_path)

In [16]:
search_export_df.head()

,query,timestamp,error,profileUrl,fullName,firstName,lastName,companyName,title,companyId,...,vmid,linkedInProfileUrl,isPremium,isOpenLink,titleDescription,pastExperienceCompanyName,pastExperienceCompanyUrl,pastExperienceCompanyTitle,pastExperienceDate,pastExperienceDuration
0,Search5 SN,2023-07-19T18:00:39.744Z,No result found,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.834Z,NaN,https://www.linkedin.com/sales/lead/ACwAABgNPU...,Dr. Edgar Noumair,Dr. Edgar,Noumair,Talal and Madiha Zein AUB Innovation Park,Mentor,71418447.0,...,ACwAABgNPUkBxBAF3JxK0VPQmHkYRnrxXzAiHjY,https://www.linkedin.com/in/ACwAABgNPUkBxBAF3J...,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.834Z,NaN,https://www.linkedin.com/sales/lead/ACwAAAHVj4...,Maureen Cusick Thomas,Maureen,Cusick Thomas,Bethesda Green,Innovation Lab Mentor,9287319.0,...,ACwAAAHVj44Brlumxgp-o7YP0oARKp_kKER4e6k,https://www.linkedin.com/in/ACwAAAHVj44Brlumxg...,True,True,The Innovation Lab Mentor Program is designed ...,NaN,NaN,NaN,NaN,NaN
3,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,NaN,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,Neil Davis,Neil,Davis,Innovation Works,Business Mentor,11787752.0,...,ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM,https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...,False,False,Innovation Works aims to reduce Baltimoreâs ...,NaN,NaN,NaN,NaN,NaN
4,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,NaN,https://www.linkedin.com/sales/lead/ACwAAACRHS...,Michael Hess,Michael,Hess,"ABS Advisory Services Group, Inc.",Chairman/CEO,NaN,...,ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8,https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...,False,False,"Manage mergers & acquisitions, corporate and r...",NaN,NaN,NaN,NaN,NaN


In [17]:
profile_export_df.head()

,error,baseUrl,timestamp,linkedinProfileUrl,email,linkedinProfile,description,headline,location,imgUrl,...,mail,companyUrl,companyUrl2,schoolUrl2,website,schoolDateRange,schoolDateRange2,birthday,facebookUrl,connectedOn
0,Not a LinkedIn Profile URL,profileUrl,2023-07-19T19:21:54.310Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,https://www.linkedin.com/sales/lead/ACwAAAHVj4...,2023-07-19T19:22:11.128Z,https://www.linkedin.com/in/maureenbcthomas/,NaN,https://www.linkedin.com/in/maureenbcthomas/,"A visionary strategist, entrepreneur and innov...",Stakeholder Engagement I Innovative Coalition ...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQGVCII...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,https://www.linkedin.com/sales/lead/ACwAABgNPU...,2023-07-19T19:22:32.106Z,https://www.linkedin.com/in/edgarnoumair/,ednoumair@hotmail.com,https://www.linkedin.com/in/edgarnoumair/,I Advise & Lead Boards & Organizations on Acti...,Data Scientist / Entrepreneur / Author / Artis...,Washington DC-Baltimore Area,https://media.licdn.com/dms/image/C5603AQFNGxE...,...,ednoumair@hotmail.com,https://www.linkedin.com/company/10522559/,https://www.linkedin.com/company/4794/,https://www.linkedin.com/company/3165/,gnoci.com,NaN,NaN,NaN,NaN,NaN
3,NaN,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,2023-07-19T19:22:51.105Z,https://www.linkedin.com/in/nrdavis/,NaN,https://www.linkedin.com/in/nrdavis/,NaN,Assisting Maryland's startup companies and ent...,"Towson, Maryland, United States",https://media.licdn.com/dms/image/D5603AQGPI--...,...,NaN,https://www.linkedin.com/company/5337426/,https://www.linkedin.com/company/1734721/,https://www.linkedin.com/company/19308/,NaN,1979 - 1982,1971 - 1975,NaN,NaN,NaN
4,NaN,https://www.linkedin.com/sales/lead/ACwAAACRHS...,2023-07-19T19:23:08.967Z,https://www.linkedin.com/in/discoveringmind/,NaN,https://www.linkedin.com/in/discoveringmind/,I help corporate and real estate teams plan an...,"Finance/M&A Entrepreneur, Director & Advisor. ...","Annapolis, Maryland, United States",https://media.licdn.com/dms/image/C4D03AQF-DLN...,...,NaN,NaN,https://www.linkedin.com/company/59839/,https://www.linkedin.com/company/4477/,NaN,1986 - 1988,Jul 2021 - Aug 2021,NaN,NaN,NaN


In [18]:
def df_to_gsheet(gsheet_name: str, df: pd.DataFrame) -> None:
    gsheet = gspread_pandas.Spread(
        gsheet_name,
        sheet="Sheet1",
        create_sheet=True,
    )
    gsheet.df_to_sheet(df, index=False)
    _LOG.info("Save to gsheet %s", gsheet_name)

In [19]:
df_to_gsheet(f"{search_name}.step1.search_export", search_export_df)
df_to_gsheet(f"{search_name}.step3.profile_export", profile_export_df)

INFO  Save to gsheet sn_test_folder.step1.search_export
INFO  Save to gsheet sn_test_folder.step3.profile_export


# Delete temp result CSVs

In [20]:
import helpers.hio as hio

In [21]:
hio.delete_file(search_result_csv_path)
_LOG.info("Delete file %s", search_result_csv_path)
hio.delete_file(profile_result_csv_path)
_LOG.info("Delete file %s", profile_result_csv_path)

INFO  Delete file ../result_csv/sn_test_folder_search_result.csv
INFO  Delete file ../result_csv/sn_test_folder_profile_result.csv
